In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from sklearn.metrics import accuracy_score
import random

#CONFIG & HYPERPARAMETERS
MODEL_NAME = "google/flan-t5-large"  #You can switch to "t5-small" for faster testing
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
MAX_LENGTH = 512
NUM_BEAMS = 4
TOP_K = 50
TOP_P = 0.95

#Simulated dataset (replace with your real dataset)
SUPPORT_TICKETS = [
    {"text": "I can't log in to my account. It says invalid credentials.", "true_tag": "login_issue"},
    {"text": "The app crashes every time I open the settings menu.", "true_tag": "app_crash"},
    {"text": "How do I reset my password? I forgot it.", "true_tag": "password_reset"},
    {"text": "Payment failed even though my card is valid.", "true_tag": "payment_issue"},
    {"text": "The search feature returns irrelevant results.", "true_tag": "search_issue"},
    {"text": "I need to update my billing address.", "true_tag": "account_update"},
    {"text": "The download button does nothing when clicked.", "true_tag": "ui_bug"},
    {"text": "Email notifications are not being sent.", "true_tag": "notification_issue"},
    {"text": "Two-factor authentication is not working.", "true_tag": "2fa_issue"},
    {"text": "I was charged twice for the same subscription.", "true_tag": "billing_dispute"},
]

#Define all possible tags
ALL_TAGS = sorted(list(set(ticket["true_tag"] for ticket in SUPPORT_TICKETS)))

#Few-shot examples (for demonstration — pick 3 random)
FEW_SHOT_EXAMPLES = random.sample(SUPPORT_TICKETS, min(3, len(SUPPORT_TICKETS)))

#MODEL SETUP
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME).to(DEVICE)

#PROMPT TEMPLATES
def zero_shot_prompt(ticket_text):
    tags_str = ", ".join(ALL_TAGS)
    return f"""Classify the following support ticket into one of these tags: {tags_str}.
Ticket: {ticket_text}
Tag:"""

def few_shot_prompt(ticket_text, examples):
    tags_str = ", ".join(ALL_TAGS)
    prompt = f"""Classify the following support ticket into one of these tags: {tags_str}.
Examples:
"""
    for ex in examples:
        prompt += f"Ticket: {ex['text']}\nTag: {ex['true_tag']}\n\n"
    prompt += f"Ticket: {ticket_text}\nTag:"
    return prompt

#INFERENCE FUNCTION
def predict_top_k_tags(prompt, k=3):
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=MAX_LENGTH).to(DEVICE)
    
#Generate multiple outputs using beam search
    outputs = model.generate(
        **inputs,
        max_new_tokens=20,
        num_beams=NUM_BEAMS * k,
        num_return_sequences=k,
        early_stopping=True,
        no_repeat_ngram_size=2,
        remove_invalid_values=True
    )
    
#Decode and clean predictions
    decoded = tokenizer.batch_decode(outputs, skip_special_tokens=True)
#Normalize: lowercase, strip, and deduplicate
    tags = list(dict.fromkeys([tag.strip().lower() for tag in decoded]))
    
#Filter to only known tags
    valid_tags = [tag for tag in tags if tag in ALL_TAGS]
    
#If we don't have k valid tags, pad with most common or random (fallback)
    while len(valid_tags) < k and len(ALL_TAGS) > 0:
        candidate = random.choice(ALL_TAGS)
        if candidate not in valid_tags:
            valid_tags.append(candidate)
    
    return valid_tags[:k]

#EVALUATION HELPER
def evaluate_predictions(true_tags, predicted_tags_list):
    """
    Simple top-1 accuracy: if true tag is in top-k predictions, count as correct.
    """
    correct = 0
    total = len(true_tags)
    for true, preds in zip(true_tags, predicted_tags_list):
        if true in preds:
            correct += 1
    return correct / total if total > 0 else 0.0

#MAIN EXECUTION
if __name__ == "__main__":
    print("🚀 Auto Tagging Support Tickets Using LLM")
    print("="*60)

#Store results
    zero_shot_predictions = []
    few_shot_predictions = []
    true_labels = [ticket["true_tag"] for ticket in SUPPORT_TICKETS]

    for ticket in SUPPORT_TICKETS:
        text = ticket["text"]

        #Zero-shot
        prompt_zs = zero_shot_prompt(text)
        top3_zs = predict_top_k_tags(prompt_zs, k=3)
        zero_shot_predictions.append(top3_zs)

        #Few-shot
        prompt_fs = few_shot_prompt(text, FEW_SHOT_EXAMPLES)
        top3_fs = predict_top_k_tags(prompt_fs, k=3)
        few_shot_predictions.append(top3_fs)

        print(f"\nTicket: {text}")
        print(f"True Tag: {ticket['true_tag']}")
        print(f"Zero-shot Top-3: {top3_zs}")
        print(f"Few-shot Top-3:  {top3_fs}")

#Evaluate
    acc_zs = evaluate_predictions(true_labels, zero_shot_predictions)
    acc_fs = evaluate_predictions(true_labels, few_shot_predictions)

    print("\n" + "="*60)
    print("📊 PERFORMANCE COMPARISON")
    print(f"Zero-shot Top-3 Accuracy: {acc_zs:.2%}")
    print(f"Few-shot Top-3 Accuracy:  {acc_fs:.2%}")

#FINE-TUNING PLACEHOLDER
    print("\n" + "="*60)
    print("💡 Fine-tuning is not implemented in this script.")
    print("To implement fine-tuning:")
    print("  1. Prepare dataset in 'input: ticket, target: tag' format")
    print("  2. Use Hugging Face Trainer with Seq2SeqTrainingArguments")
    print("  3. Train model and replace inference model above")
    print("Example fine-tuning guide: https://huggingface.co/docs/transformers/training")

    print("\n✅ Done.")

tokenizer_config.json: 0.00B [00:00, ?B/s]

C:\Users\daudr\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\daudr\.cache\huggingface\hub\models--google--flan-t5-large. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

🚀 Auto Tagging Support Tickets Using LLM

Ticket: I can't log in to my account. It says invalid credentials.
True Tag: login_issue
Zero-shot Top-3: ['login_issue', 'account_update', 'password_reset']
Few-shot Top-3:  ['login_issue', 'account_update', 'password_reset']

Ticket: The app crashes every time I open the settings menu.
True Tag: app_crash
Zero-shot Top-3: ['app_crash', '2fa_issue', 'ui_bug']
Few-shot Top-3:  ['app_crash', '2fa_issue', 'login_issue']

Ticket: How do I reset my password? I forgot it.
True Tag: password_reset
Zero-shot Top-3: ['password_reset', 'login_issue', 'account_update']
Few-shot Top-3:  ['password_reset', 'login_issue', 'account_update']

Ticket: Payment failed even though my card is valid.
True Tag: payment_issue
Zero-shot Top-3: ['payment_issue', 'billing_dispute', '2fa_issue']
Few-shot Top-3:  ['payment_issue', 'billing_dispute', '2fa_issue']

Ticket: The search feature returns irrelevant results.
True Tag: search_issue
Zero-shot Top-3: ['search_issue'